In [ ]:
# PSSSS! READ ME! 
# Key Bea = 30f6c00168msh1b42af1260e2970p1cb797jsn775f4f1bf53e
# Key Bea2 = 7da7d1d9f2mshd70f16dedff8827p11e4cejsnc105329a0d9c
# Key Ceci = dab5a0167dmsh4ab1543f0fd8e0ap1d9b21jsnd66b36d85213
# Key Carla = f600d33197mshe21c869c19877bbp16c957jsn8dea9e68b50b
# -------------------------------- (these keys must be hiden)

# purpose of the code: extract data from the MoviesDatabase in Rapid API, process the data and create a csv with the output data
# API documentation https://rapidapi.com/SAdrian/api/moviesdatabase

In [ ]:
# IMPORTS

import requests
import pandas as pd

In [ ]:
# FUNCTIONS DEFINITION

# function for calling the API
def api_call(url, params):
    headers = {
        "x-rapidapi-key": api_key, 
        "x-rapidapi-host": "moviesdatabase.p.rapidapi.com"
    }
    
    call = requests.get(url, headers=headers, params=params)
    print(f"\nThe API call gave us a response of type: {call.status_code}")
    if call.status_code != 200:
        print(f"\nThe reason for the failure is: {call.reason}")
        return None
    else:
        response = call.json()
        return response


# function to process the API response: extracts the necessary information and generates a list of tuples as output. It requires the API response to be given as an argument.
def process_response(response, columns_list, index, genre):
     
    processed_data = []             # create a list to store extracted data
    
    for item in response:               # iterate over each element in the list, extract the data needed, store it in a list per item, and then append the item list to the general list (processed_data) 
        
        item_data = []
        
        item_data.append(item["titleType"].get("text"))
        
        item_data.append(item["titleText"].get("text"))
        
        try:
            item_data.append(item["releaseYear"].get("year"))
        except:
            item_data.append("none")
        
        try:
            item_data.append(item["releaseDate"].get("month"))
        except:
            item_data.append("none")
        
        item_data.append(item["id"])

        genre=genre
        if genre:
            item_data.append(genre)
        
        processed_data.append(tuple(item_data))

    print ("\nData extracted ✅")

    # create DataFrame
    columns_list = columns_list
    df_procesed_data = pd.DataFrame(processed_data, columns=columns_list)
    # set index
    index = index
    df_procesed_data.set_index(index, inplace=True)
    
    print ("\nDataFrame created ✅")
    
    return df_procesed_data

In [ ]:
# CODE EXECUTION ⬇️⬇️⬇️ USE THIS TO CONNECT TO ENDPOINT TITLES - Title (when you don't know the movies' ids)

api_key = "f600d33197mshe21c869c19877bbp16c957jsn8dea9e68b50b"


# 1) call api_call function

# define params ⚠️ CHANGE THE START PAGE AND GENRE TO CHANGE THE API REQUEST
genre = "Drama"     # "Action", "Comedy", "Drama"
api_start_page = 1
api_end_page = 2

for i in range (api_start_page, api_end_page):                
    print (f"\n<< Iteration {i}>>")
    raw_movies_data = []
    title_types = ["movie", "short"]
    params = None
    for type in title_types: # call function for each title type
        url = f'https://moviesdatabase.p.rapidapi.com/titles?genre={genre}&startYear=2010&titleType={type}&page={i}&endYear=2024&limit=50'
        response = api_call(url, params=params)
        data = response ["results"] 
        raw_movies_data.extend(data)

    # 2) process data

    # define params
    index =  "id_imdb"
    columns_list = ["type", "name", "release_year", "release_month", "id_imdb", "genre"]

    # call function
    response = raw_movies_data
    df_movies_data = process_response (response, columns_list, index, genre) # en lugar de una respuesta de tipo json, le voy a dar directamente la lista


    # 3) save to csv
    df_movies_data.to_csv(f"data/api/list_{genre.lower()}_range_{i}_{i+1}_API.csv")
    print ("\n.csv file created ✅")

    df_movies_data


In [ ]:
# CODE EXECUTION ⬇️⬇️⬇️ USE THIS TO CONNECT TO ENDPOINT TITLES - By List of Id's (when you know the movies' ids)

api_key = "f600d33197mshe21c869c19877bbp16c957jsn8dea9e68b50b"

# 1) call api function

# define params
url = "https://moviesdatabase.p.rapidapi.com/titles/x/titles-by-ids"
# imdb_ids of all the movies that won oscars 

# this list contains the imdb_id of the TOP250 movies in groups of 25 ids per element on the list

top_250_ids = ['tt0111161,tt0068646,tt0468569,tt0071562,tt0050083,tt0108052,tt0167260,tt0110912,tt0120737,tt0060196,tt0109830,tt0167261,tt0137523,tt1375666,tt0080684,tt0133093,tt0099685,tt0073486,tt0114369,tt0816692,tt0038650,tt0047478,tt0102926,tt0120815,tt0317248',
 'tt0118799,tt0120689,tt0103064,tt0076759,tt0088763,tt0245429,tt0253474,tt6751668,tt0054215,tt15239678,tt0172495,tt0110357,tt9362722,tt0407887,tt0110413,tt0120586,tt2582802,tt0482571,tt0095327,tt0056058,tt0114814,tt0034583,tt1675434,tt0095765,tt0027977',
 'tt0047396,tt0064116,tt0078748,tt0021749,tt1853728,tt0078788,tt0209144,tt0910970,tt0082971,tt23849204,tt0405094,tt0043014,tt4154756,tt0050825,tt4633694,tt0051201,tt0081505,tt0032553,tt0090605,tt0361748,tt1345836,tt2380307,tt0057012,tt0086879,tt0364569',
 'tt0169547,tt0114709,tt4154796,tt0082096,tt0112573,tt0119217,tt0119698,tt7286456,tt5311514,tt0057565,tt1187043,tt0087843,tt0045152,tt8267604,tt0091251,tt0180093,tt0435761,tt0086190,tt0338013,tt2106476,tt0062622,tt0105236,tt0044741,tt0056172,tt15398776',
 'tt0053604,tt1255953,tt0053125,tt0033467,tt0086250,tt0036775,tt0022100,tt0052357,tt0093058,tt0211915,tt0113277,tt1049413,tt0066921,tt0056592,tt1832382,tt0070735,tt0095016,tt0097576,tt0986264,tt0017136,tt0208092,tt8579674,tt0119488,tt0040522,tt8503618',
 'tt0075314,tt0363163,tt5074352,tt0372784,tt0059578,tt0993846,tt0053291,tt6966692,tt0012349,tt10272386,tt0055031,tt0120382,tt0042192,tt1130884,tt1745960,tt0469494,tt0112641,tt0107290,tt0089881,tt0167404,tt0457430,tt0105695,tt0477348,tt0268978,tt0084787',
 'tt0266697,tt0040897,tt0055630,tt0071853,tt0057115,tt0266543,tt1392214,tt0347149,tt0042876,tt0080678,tt0046912,tt0071315,tt0031381,tt0120735,tt0434409,tt1305806,tt2096673,tt0081398,tt5027774,tt0117951,tt0050212,tt4729430,tt0116282,tt0264464,tt10872600',
 'tt1291584,tt1205489,tt1201607,tt0405159,tt0096283,tt0118849,tt2024544,tt0052618,tt0083658,tt0112471,tt1392190,tt0072684,tt2278388,tt2267998,tt2119532,tt0097165,tt0353969,tt0015864,tt0107207,tt0198781,tt3011894,tt0077416,tt0017925,tt0015324,tt0047296',
 'tt0073195,tt0892769,tt0978762,tt0382932,tt0041959,tt0046268,tt0050986,tt0031679,tt1950186,tt0046438,tt0118715,tt0075148,tt0050976,tt3170832,tt3315342,tt1895587,tt0395169,tt0088247,tt0091763,tt0113247,tt0381681,tt0019254,tt0325980,tt0036868,tt15097216',
 'tt0070047,tt1979320,tt0074958,tt0032138,tt0092005,tt0317705,tt1028532,tt0059742,tt4016934,tt0476735,tt0758758,tt0035446,tt0058946,tt0032551,tt0107048,tt0245712,tt0129167,tt0032976,tt0061512,tt1454029,tt0025316,tt0103639,tt11032374,tt0099348,tt0087884']

for i, ids_group in enumerate (top_250_ids, 1):

    print (f"<< Iteration {i}>>")
    params = {"idsList":ids_group}

    # call function
    response = api_call(url, params=params)
    response = response ["results"]     # access "results" key values, where data is conteined. Data is in a list format.

    # 2) process data

    # define params
    columns_list = ["type", "name", "release_year", "release_month", "id_imdb"]
    index = "id_imdb"
    genre= None

    # call function
    df_movies_data = process_response (response, columns_list, index, genre)


    # 3) save as csv

    file_name = f"top250_{i}"
    df_movies_data.to_csv(f"data/api/list_{file_name}_API.csv") # ⚠️ IMPORTANT! Review the name and path for the csv file
    print ("\n.csv file created ✅")

    df_movies_data

In [ ]:
# PREP-WORK for TOP250 movies (THIS IS A BONUS WE MIGHT NO IMPLEMENT)
# transform the string of all top250 ids into groups of 25 ids each, so it can be used to call the api
top250_ids_as_one_string = "tt0111161,tt0068646,tt0468569,tt0071562,tt0050083,tt0108052,tt0167260,tt0110912,tt0120737,tt0060196,tt0109830,tt0167261,tt0137523,tt1375666,tt0080684,tt0133093,tt0099685,tt0073486,tt0114369,tt0816692,tt0038650,tt0047478,tt0102926,tt0120815,tt0317248,tt0118799,tt0120689,tt0103064,tt0076759,tt0088763,tt0245429,tt0253474,tt6751668,tt0054215,tt15239678,tt0172495,tt0110357,tt9362722,tt0407887,tt0110413,tt0120586,tt2582802,tt0482571,tt0095327,tt0056058,tt0114814,tt0034583,tt1675434,tt0095765,tt0027977,tt0047396,tt0064116,tt0078748,tt0021749,tt1853728,tt0078788,tt0209144,tt0910970,tt0082971,tt23849204,tt0405094,tt0043014,tt4154756,tt0050825,tt4633694,tt0051201,tt0081505,tt0032553,tt0090605,tt0361748,tt1345836,tt2380307,tt0057012,tt0086879,tt0364569,tt0169547,tt0114709,tt4154796,tt0082096,tt0112573,tt0119217,tt0119698,tt7286456,tt5311514,tt0057565,tt1187043,tt0087843,tt0045152,tt8267604,tt0091251,tt0180093,tt0435761,tt0086190,tt0338013,tt2106476,tt0062622,tt0105236,tt0044741,tt0056172,tt15398776,tt0053604,tt1255953,tt0053125,tt0033467,tt0086250,tt0036775,tt0022100,tt0052357,tt0093058,tt0211915,tt0113277,tt1049413,tt0066921,tt0056592,tt1832382,tt0070735,tt0095016,tt0097576,tt0986264,tt0017136,tt0208092,tt8579674,tt0119488,tt0040522,tt8503618,tt0075314,tt0363163,tt5074352,tt0372784,tt0059578,tt0993846,tt0053291,tt6966692,tt0012349,tt10272386,tt0055031,tt0120382,tt0042192,tt1130884,tt1745960,tt0469494,tt0112641,tt0107290,tt0089881,tt0167404,tt0457430,tt0105695,tt0477348,tt0268978,tt0084787,tt0266697,tt0040897,tt0055630,tt0071853,tt0057115,tt0266543,tt1392214,tt0347149,tt0042876,tt0080678,tt0046912,tt0071315,tt0031381,tt0120735,tt0434409,tt1305806,tt2096673,tt0081398,tt5027774,tt0117951,tt0050212,tt4729430,tt0116282,tt0264464,tt10872600,tt1291584,tt1205489,tt1201607,tt0405159,tt0096283,tt0118849,tt2024544,tt0052618,tt0083658,tt0112471,tt1392190,tt0072684,tt2278388,tt2267998,tt2119532,tt0097165,tt0353969,tt0015864,tt0107207,tt0198781,tt3011894,tt0077416,tt0017925,tt0015324,tt0047296,tt0073195,tt0892769,tt0978762,tt0382932,tt0041959,tt0046268,tt0050986,tt0031679,tt1950186,tt0046438,tt0118715,tt0075148,tt0050976,tt3170832,tt3315342,tt1895587,tt0395169,tt0088247,tt0091763,tt0113247,tt0381681,tt0019254,tt0325980,tt0036868,tt15097216,tt0070047,tt1979320,tt0074958,tt0032138,tt0092005,tt0317705,tt1028532,tt0059742,tt4016934,tt0476735,tt0758758,tt0035446,tt0058946,tt0032551,tt0107048,tt0245712,tt0129167,tt0032976,tt0061512,tt1454029,tt0025316,tt0103639,tt11032374,tt0099348,tt0087884"

top250_ids_as_list = top250_ids_as_one_string.split(",")

top_250_ids = []
for i in range (0,250,25):
    list_25_ids = top250_ids_as_list [i: i+25]
    string_25_ids = ",".join(list25)
    top_250_ids.append(string_25_ids)

top_250_ids
